<a href="https://colab.research.google.com/github/croco22/CapstoneProjectTDS/blob/philipp/notebooks/ExtraTask2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extra Task 2: Continuous Text Evaluation

## Imports and Setup

In [7]:
import json
import time
from google.colab import userdata
import google.generativeai as genai
import pandas as pd
from transformers import pipeline


# API Setup
genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel('gemini-2.0-flash-exp')
userdata.get('HF_TOKEN')


def generate_text(prompt):
    try:
        response = model.generate_content(
            prompt,
            generation_config = genai.GenerationConfig(
                temperature=2.0,
            )
        )
        time.sleep(5) # Avoid exceeding API limits
        return response.text.strip()
    except Exception as e:
        exit("Error during API call: ", e)

In [10]:
# Read dataset files
url = 'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/philipp/qa_dataset_continuous.json' # Todo: Change branch
continuous_data = pd.read_json(url)

url2 = 'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/main/qa_dataset.json'
data = pd.read_json(url2)

continuous_data.head()

,text,questions
0,"In regards to the customer group, it's importa...","[{'question': 'Customer group', 'intended_answ..."
1,"Nope, definitely not, please don’t send me any...",[{'question': 'Would you like to receive marke...
2,"And finally, and this is something I always ap...",[{'question': 'How could this app better suppo...
3,"Also, sometimes I think it’s just a tad slow t...",[{'question': 'How could we improve your exper...
4,"Finally, something I'm struggling with quite a...",[{'question': 'What specific challenges are yo...


In [8]:
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2", tokenizer="deepset/roberta-base-squad2")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


In [14]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import torch

# Datensatz laden und vorbereiten
texts = ["Das Unternehmen plant, die Produktion um 20 % zu steigern."]
questions = ["Wie viel Prozent beträgt die geplante Produktionssteigerung?", "Welche neuen Produkte werden eingeführt?"]
labels = [1, 0]  # Beispielhafte Labels

# Tokenizer und Modell laden
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Daten vorbereiten
class TextQuestionDataset(Dataset):
    def __init__(self, texts, questions, labels, tokenizer):
        self.texts = texts
        self.questions = questions
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]
        question = self.questions[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, question, padding='max_length', truncation=True, return_tensors="pt")
        return {**encoding, 'label': torch.tensor(label, dtype=torch.long)}

dataset = TextQuestionDataset(texts, questions, labels, tokenizer)

# Trainingsprozess starten
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
for batch in dataloader:
    outputs = model(**batch)
    loss = outputs.loss
    logits = outputs.logits


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


IndexError: list index out of range